In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from cmath import cos, sin, pi

In [ ]:
# Load the raw EEG data
eeg_data = pd.read_csv("data/Liuyin-5April/Morning/wet_17hz_0504_clean_ExG.csv")

# Set the sampling rate and the filter cutoff frequencies
fs = 250
low_cutoff = 6
high_cutoff = 30

#Create two dataframes from eeg_data the first is the first column which is the timestamp and the second is the remaiing columns which are the EEG data
eeg_data_timestamp = eeg_data.iloc[:,0]
eeg_data_eeg = eeg_data.iloc[:,1:]

# Create an array of the EEG data
eeg_array = eeg_data_eeg.to_numpy() # 40672 rows x 8 columns (channels)

eeg_array = eeg_array.T # 8 rows (channels) x 40672 columns (samples)




In [ ]:

def rls_filter(eeg_data, target_freq, lambda_=0.98):
    """
    Implements the RLS (Recursive Least Squares) adaptive filter for SSVEP detection.

    Args:
        eeg_data (ndarray): Input EEG data of shape (n_channels, n_samples).
        target_freq (float): Target frequency of the SSVEP stimulus.
        lambda_ (float, optional): Forgetting factor. Default is 0.98.

    Returns:
        ndarray: Filtered data of shape (n_channels, n_samples).
    """
    n_samples = eeg_data.shape[1]
    r1 = sin (2*pi*target_freq/fs)
    r1 = r1[0:n_samples]
    r2 = cos (2*pi*target_freq/fs)
    r2 = r2[0:n_samples]
    # Initialize filter coefficients, P matrix, and estimated output
    w = np.zeros((2, 1)) # 2 filter coefficients hc and hs 
    P = np.eye(2) / 0.001 # P matrix is a 2x2 identity matrix
    filtered_data = np.zeros_like(eeg_data)

    # Apply RLS filter
    for t in range(n_samples):
        k = np.dot(P, x) / (lambda_ + np.dot(np.dot(x.T, P), x))  # Kalman gain
        e = eeg_data[:, t] - y  # error signal
        w = w + k * e  # filter coefficients update
        P = (1 / lambda_) * (P - np.dot(np.dot(k, x.T), P))  # P matrix update
        
        x = eeg_data[:, t].reshape(-1, 1)  # current input sample
        y = np.dot(w.T, x)  # estimated output
        filtered_data[:, t] = y.flatten()  # store filtered output

    return filtered_data

In [ ]:
rls_data = rls_filter(eeg_array, 13)

print(rls_data.shape)

#Plot the data for each channel seperately before and after filtering
# for i in range(8):
#     plt.figure()
#     plt.plot(eeg_array[i,:])
#     plt.title("Channel "+str(i+1)+" before filtering")
#     plt.figure()
#     plt.plot(rls_data[i,:])
#     plt.title("Channel "+str(i+1)+" after filtering")
#     plt.show

#Plot the power spectrum of the data before and after filtering ranging from 0 to 50 Hz
for i in range(8):
    plt.figure()
    plt.psd(eeg_array[i,:], Fs=250, NFFT=1024)
    plt.xlim(6,40)
    plt.title("Channel "+str(i+1)+" before filtering")    
    plt.figure()
    plt.psd(rls_data[i,:], Fs=250, NFFT=1024)
    plt.xlim(6,40)
    plt.title("Channel "+str(i+1)+" after filtering")
    plt.show

